In [1]:
import cn2an
from operation import chinese_to_int

In [58]:

replaceChineseNumbers_test = {
    '一萬八千': 18000,
    '二十三': 23,
    '100,000': 100000,
    '1百萬': 1000000,
    '一百萬': 1000000,
    '一千零一十': 1010,
    '一百零一': 101,
    '二三': 23,
    '九九': 99,
    '3.5萬': 35000,
    "十四": 14,
    "四十": 40,
    "6拾": 60,
    "6拾萬": 600000,
    "拾萬": 100000,
    "10萬": 100000,
    "一百萬": 1000000,
    "100萬": 1000000,
    "拾1": 11,
    "2佰8拾6万3仟": 2863000,
    "三千八十":3080
}

for key, number in replaceChineseNumbers_test.items():
    key = key.replace(',', '')
    translate_number = cn2an.cn2an(key, 'smart')
    if translate_number != number:
        print(f"錯誤 {key} => {translate_number} (正確答案: {number})")

# 日期

In [67]:
import re
import cn2an

# fact
line_list = [
    '處拘役零年零月肆拾日', 
    '處拘役零年零月肆拾日',
    '有期徒刑零年貳月',
    "又犯非法製造子彈罪，處有期徒刑壹年，併科罰金新臺幣壹萬元，罰金如易服勞役，以新臺幣壹仟元折算壹日。有期徒刑部分應執行有期徒刑伍年陸月，罰金部分應執行新臺幣參萬伍仟元，罰金如易服勞役，以新臺幣壹仟元折算壹日。",
    "蕭迪晃駕駛動力交通工具而吐氣所含酒精濃度達每公升零點二五毫克以上，處有期徒刑參月，如易科罰金，以新臺幣壹仟元折算壹日。",
    "林家緯犯三人以上共同冒用政府機關及公務員名義犯詐欺取財罪，處有期徒刑以壹年肆月。未扣案之林家緯犯罪所得新臺幣貳仟元沒收，於全部或一部不能沒收或不宜執行沒收時，追徵其價額。",
    "鄭坤盈犯轉讓禁藥罪，累犯，處有期徒刑陸月。又犯如附表所示之罪，各處如附表所示之刑。有期徒刑逾陸月部分，應執行有期徒刑拾壹年。扣案手機壹支（含SIM 卡壹張）、磅秤壹個均沒收。未扣案犯罪所得新臺幣貳萬叁仟壹佰元沒收，於全部或一部不能沒收或不宜執行沒收時，追徵其價額。",
    "庚○○犯森林法第五十二條第三項、第一項第一款、第四款、第六款之竊取森林主產物貴重木罪，處有期徒刑壹年拾月，併科罰金新臺幣貳佰捌拾陸萬參仟捌拾元，罰金如易服勞役，以罰金總額與壹年之日數比例折算。扣案如附表編號7 至14、16至17、22所示之物，均沒收。己○○犯森林法第五十二條第三項、第一項第一款、第四款、第六款之竊取森林主產物貴重木罪，累犯，處有期徒刑壹年捌月，併科罰金新臺幣貳佰陸拾貳萬肆仟肆佰玖拾元，罰金如易服勞役，以罰金總額與壹年之日數比例折算。扣案如附表編號7 至14、16至17、 22所示之物，均沒收。丙○○犯森林法第五十二條第三項、第一項第一款、第四款、第六款之竊取森林主產物貴重木罪，累犯，處有期徒刑壹年?月，併科罰金新臺幣貳佰參拾捌萬伍仟玖佰元，罰金如易服勞役，以罰金總額與壹年之日數比例折算。扣案如附表編號7 至14、16至17、22所示之物，均沒收。甲○○犯森林法第五十二條第三項、第一項第一款、第四款、第六款之竊取森林主產物貴重木罪，處有期徒刑柒月，併科罰金新臺幣壹佰貳拾萬元，罰金如易服勞役，以罰金總額與壹年之日數比例折算。扣案如附表編號7 至14、16至17、22所示之物，均沒收。",
    '高榮利犯恐嚇危害安全罪，共貳罪，各處拘役肆拾日，如易科罰金，均以新臺幣壹仟元折算壹日；又犯傷害罪，處拘役伍拾日，如易科罰金，以新臺幣壹仟元折算壹日。應執行拘役壹佰日，如易科罰金，以新臺幣壹仟元折算壹日。',
    "吳良旭、鄭文旗犯傷害罪，各處拘役貳拾日，如易科罰金，均以新臺幣壹仟元折算壹日；又犯公然侮辱罪，各處拘役壹拾日，如易科罰金，均以新臺幣壹仟元折算壹日。均應執行拘役貳拾伍日，如易科罰金，均以新臺幣壹仟元折算壹日。"
]

total_imprisonment = 0


def parse_time(time_str, line):
    # 解析時間字符串，返回年、月、日的數值
    try:
        return int(cn2an.cn2an(chinese_to_int(time_str) if time_str != '' else 0, 'smart'))
    except ValueError as e:
        print(f"!-> 相關匹配: {time_str}, 錯誤詳情: {e}")
        return 0

for line in line_list:
    
    print("====", line)

    imprisonments = []
    total_imprisonment = 0  

    if '年' in line or '月' in line or '日' in line:
        
        # 合併正則表達式
        pattern = re.compile(r'(有期徒刑|拘役)(?:(?:以)?([\u4e00-\u9fff]{1,2})年)?(?:([\u4e00-\u9fff]{1,2})月)?(?:又?([\u4e00-\u9fff]{1,2})日)?')
        matches = pattern.findall(line)
        
        for match in matches:
            sentence_type, years, months, days = match

            years_val = parse_time(years, line)
            months_val = parse_time(months, line)
            days_val = parse_time(days, line)

            if years_val != 0 or months_val != 0 or days_val != 0:
                imprisonment_str = f"{sentence_type}{years_val}年{months_val}月{days_val}日" if days else f"{sentence_type}{years_val}年{months_val}月"
                print(imprisonment_str)
            

==== 處拘役零年零月肆拾日
拘役0年0月40日
==== 處拘役零年零月肆拾日
拘役0年0月40日
==== 有期徒刑零年貳月
有期徒刑0年2月
==== 又犯非法製造子彈罪，處有期徒刑壹年，併科罰金新臺幣壹萬元，罰金如易服勞役，以新臺幣壹仟元折算壹日。有期徒刑部分應執行有期徒刑伍年陸月，罰金部分應執行新臺幣參萬伍仟元，罰金如易服勞役，以新臺幣壹仟元折算壹日。
有期徒刑1年0月
有期徒刑5年6月
==== 蕭迪晃駕駛動力交通工具而吐氣所含酒精濃度達每公升零點二五毫克以上，處有期徒刑參月，如易科罰金，以新臺幣壹仟元折算壹日。
有期徒刑0年3月
==== 林家緯犯三人以上共同冒用政府機關及公務員名義犯詐欺取財罪，處有期徒刑以壹年肆月。未扣案之林家緯犯罪所得新臺幣貳仟元沒收，於全部或一部不能沒收或不宜執行沒收時，追徵其價額。
有期徒刑1年4月
==== 鄭坤盈犯轉讓禁藥罪，累犯，處有期徒刑陸月。又犯如附表所示之罪，各處如附表所示之刑。有期徒刑逾陸月部分，應執行有期徒刑拾壹年。扣案手機壹支（含SIM 卡壹張）、磅秤壹個均沒收。未扣案犯罪所得新臺幣貳萬叁仟壹佰元沒收，於全部或一部不能沒收或不宜執行沒收時，追徵其價額。
有期徒刑0年6月
!-> 相關匹配: 逾陸, 錯誤詳情: 当前为smart模式，输入的数据不在转化范围内：逾！
有期徒刑11年0月
==== 庚○○犯森林法第五十二條第三項、第一項第一款、第四款、第六款之竊取森林主產物貴重木罪，處有期徒刑壹年拾月，併科罰金新臺幣貳佰捌拾陸萬參仟捌拾元，罰金如易服勞役，以罰金總額與壹年之日數比例折算。扣案如附表編號7 至14、16至17、22所示之物，均沒收。己○○犯森林法第五十二條第三項、第一項第一款、第四款、第六款之竊取森林主產物貴重木罪，累犯，處有期徒刑壹年捌月，併科罰金新臺幣貳佰陸拾貳萬肆仟肆佰玖拾元，罰金如易服勞役，以罰金總額與壹年之日數比例折算。扣案如附表編號7 至14、16至17、 22所示之物，均沒收。丙○○犯森林法第五十二條第三項、第一項第一款、第四款、第六款之竊取森林主產物貴重木罪，累犯，處有期徒刑壹年?月，併科罰金新臺幣貳佰參拾捌萬伍仟玖佰元，罰金如易服勞役，以罰金總額與壹年之日數比例折算。扣案如附表編號7 至14、16至17、22所示之物，均沒收。甲○○犯森林法第五十二條第三項、第一項第一款、第四款、第

In [37]:
line_list = [
    "陳阿順犯森林法第五十二條第三項、第一項第四款、第六款之結夥二人以上為搬運贓物、使用車輛竊取森林主產物貴重木罪，處有期徒刑壹年參月，併科罰金新臺幣貳佰零肆萬玖仟柒佰參拾元，罰金如易服勞役，以罰金總額與壹年之日數比例折算。未扣案如附表所示之犯罪所得均沒收，於全部或一部不能沒收或不宜執行沒收時，各追徵其價額。",
    "姜瀚承犯非法持有子彈罪，處有期徒刑參月，併科罰金新臺幣壹萬伍仟萬元，有期徒刑如易科罰金、罰金如易服勞役，均以新臺幣壹仟元折算壹日；又犯竊盜未遂罪，處拘役陸日，如易科罰金，以新臺幣壹仟元折算壹日。扣案如附表所示驗餘非制式子彈肆顆均沒收。",
    "庚○○犯森林法第五十二條第三項、第一項第一款、第四款、第六款之竊取森林主產物貴重木罪，處有期徒刑壹年拾月，併科罰金新臺幣貳佰捌拾陸萬參仟捌拾元，罰金如易服勞役，以罰金總額與壹年之日數比例折算。扣案如附表編號7 至14、16至17、22所示之物，均沒收。己○○犯森林法第五十二條第三項、第一項第一款、第四款、第六款之竊取森林主產物貴重木罪，累犯，處有期徒刑壹年捌月，併科罰金新臺幣貳佰陸拾貳萬肆仟肆佰玖拾元，罰金如易服勞役，以罰金總額與壹年之日數比例折算。扣案如附表編號7 至14、16至17、 22所示之物，均沒收。丙○○犯森林法第五十二條第三項、第一項第一款、第四款、第六款之竊取森林主產物貴重木罪，累犯，處有期徒刑壹年?月，併科罰金新臺幣貳佰參拾捌萬伍仟玖佰元，罰金如易服勞役，以罰金總額與壹年之日數比例折算。扣案如附表編號7 至14、16至17、22所示之物，均沒收。甲○○犯森林法第五十二條第三項、第一項第一款、第四款、第六款之竊取森林主產物貴重木罪，處有期徒刑柒月，併科罰金新臺幣壹佰貳拾萬元，罰金如易服勞役，以罰金總額與壹年之日數比例折算。扣案如附表編號7 至14、16至17、22所示之物，均沒收。"""
]

pattern = re.compile(r"罰金新臺幣([\u4e00-\u9fff]+)元")


for line in line_list:
    print("====", line)
    
    amount_list = []
    matches = pattern.findall(line)  # 使用 findall 來找到所有匹配
    for match in matches:
        try:
            regular_match = cn2an.cn2an(chinese_to_int(match), 'smart')
        except Exception as e:
            print(f"!-> 相關匹配: {match}, 錯誤詳情: {e}")


==== 陳阿順犯森林法第五十二條第三項、第一項第四款、第六款之結夥二人以上為搬運贓物、使用車輛竊取森林主產物貴重木罪，處有期徒刑壹年參月，併科罰金新臺幣貳佰零肆萬玖仟柒佰參拾元，罰金如易服勞役，以罰金總額與壹年之日數比例折算。未扣案如附表所示之犯罪所得均沒收，於全部或一部不能沒收或不宜執行沒收時，各追徵其價額。
==== 姜瀚承犯非法持有子彈罪，處有期徒刑參月，併科罰金新臺幣壹萬伍仟萬元，有期徒刑如易科罰金、罰金如易服勞役，均以新臺幣壹仟元折算壹日；又犯竊盜未遂罪，處拘役陸日，如易科罰金，以新臺幣壹仟元折算壹日。扣案如附表所示驗餘非制式子彈肆顆均沒收。
!-> 相關匹配: 壹萬伍仟萬, 錯誤詳情: 不符合格式的数据：1万5仟万
==== 庚○○犯森林法第五十二條第三項、第一項第一款、第四款、第六款之竊取森林主產物貴重木罪，處有期徒刑壹年拾月，併科罰金新臺幣貳佰捌拾陸萬參仟捌拾元，罰金如易服勞役，以罰金總額與壹年之日數比例折算。扣案如附表編號7 至14、16至17、22所示之物，均沒收。己○○犯森林法第五十二條第三項、第一項第一款、第四款、第六款之竊取森林主產物貴重木罪，累犯，處有期徒刑壹年捌月，併科罰金新臺幣貳佰陸拾貳萬肆仟肆佰玖拾元，罰金如易服勞役，以罰金總額與壹年之日數比例折算。扣案如附表編號7 至14、16至17、 22所示之物，均沒收。丙○○犯森林法第五十二條第三項、第一項第一款、第四款、第六款之竊取森林主產物貴重木罪，累犯，處有期徒刑壹年?月，併科罰金新臺幣貳佰參拾捌萬伍仟玖佰元，罰金如易服勞役，以罰金總額與壹年之日數比例折算。扣案如附表編號7 至14、16至17、22所示之物，均沒收。甲○○犯森林法第五十二條第三項、第一項第一款、第四款、第六款之竊取森林主產物貴重木罪，處有期徒刑柒月，併科罰金新臺幣壹佰貳拾萬元，罰金如易服勞役，以罰金總額與壹年之日數比例折算。扣案如附表編號7 至14、16至17、22所示之物，均沒收。
!-> 相關匹配: 貳佰捌拾陸萬參仟捌拾, 錯誤詳情: 不符合格式的数据：2佰8拾6万3仟8拾


ValueError: 不符合格式的数据：2佰8拾6万3仟08十

In [65]:
from enum import Enum

class REASON(Enum):
    未抓取成功 = -1
    無罪 = 0
    有罪 = 1
    免刑 = 2
    不受理 = 3
    
REASON(-1).name, 

('未抓取成功',)